In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import dxchange

In [ ]:

n = 512  # object size in each dimension
pad = 256
npos= 16

detector_pixelsize = 3.03751e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
sx0 = 1.286e-3
z1 = np.tile(5.5*1e-3-sx0, [npos]) # 5.5
z2 = focusToDetectorDistance-z1
distances = (z1*z2)/focusToDetectorDistance
magnifications = focusToDetectorDistance/z1
voxelsize = np.abs(detector_pixelsize/magnifications[0]*2048/n)  # object voxel size


In [ ]:
rec={}
rec_prb={}
err={}
l={}
l['v1']='No zero-padding V1, probe fixed'
l['v1_prb_']='No zero-padding V1, probe reconstructed'
l['v2']='No zero-padding V2, probe fixed'
l['v2_prb']='No zero-padding V2, probe reconstructed'
l['noe']='Zero-padding, probe fixed'
l['noe_prb']='Zero-padding, probe reconstructed'

iter = 1024
for i,t in enumerate(['v1','v2','noe','v1_prb_','v2_prb','noe_prb']):
    path = f'/data/vnikitin/modeling/siemens_recn_{t}{n}_{pad}'
    amp=dxchange.read_tiff(f'{path}/crec_code_abs/{iter}.tiff')
    angle=dxchange.read_tiff(f'{path}/crec_code_angle/{iter}.tiff') 
    rec[f'{t}'] = amp*np.exp(1j*angle)

    amp=dxchange.read_tiff(f'{path}/crec_prb_abs/{iter}.tiff')
    angle=dxchange.read_tiff(f'{path}/crec_prb_angle/{iter}.tiff') 
    rec_prb[f'{t}'] = amp*np.exp(1j*angle)
    
    err[f'{t}']=np.load(f'{path}/err.npy')[:iter]

for i,t in enumerate(['v1','v2','noe','v1_prb_','v2_prb','noe_prb']):
    plt.plot(np.arange(0,iter,32),err[f'{t}'][::32],label=l[f'{t}'])

plt.legend()
plt.yscale('log')
# plt.show()
plt.savefig('conv',dpi=300)
# plt.plot()
    


In [ ]:
iter=256
fig, ax = plt.subplots(2,3, figsize=(15,8))
fig.suptitle(f'reconstructed object phase, iter {iter}', fontsize=15)
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.title('ss')

for i,v in enumerate(['v1','v2','noe','v1_prb_','v2_prb','noe_prb']):
    im=ax[np.unravel_index(i, (2,3))].imshow(np.angle(rec[v]),cmap='gray',vmax=0.5,vmin=-1)
    divider = make_axes_locatable(ax[np.unravel_index(i, (2,3))])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    ax[np.unravel_index(i, (2,3))].set_title(l[v])
    left, bottom, width, height = [0.123+i%3*0.274, 0.53-i//3*0.42, 0.14, 0.2]
    ax2 = fig.add_axes([left, bottom, width, height])
    ax2.imshow(np.angle(rec[v][350:550,650:850]),cmap='gray',vmax=0.5,vmin=-1)
    ax2.axis('off')
    
    fig.colorbar(im, cax=cax, orientation='vertical')
plt.savefig(f'rec_angle_{iter}',dpi=450)

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(15,8))
fig.suptitle(f'reconstructed object amp, iter {iter}', fontsize=15)
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.title('ss')

for i,v in enumerate(['v1','v2','noe','v1_prb_','v2_prb','noe_prb']):
    im=ax[np.unravel_index(i, (2,3))].imshow(np.abs(rec[v]),cmap='gray',vmax=1.05,vmin=0.95)
    divider = make_axes_locatable(ax[np.unravel_index(i, (2,3))])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    ax[np.unravel_index(i, (2,3))].set_title(l[v])
    left, bottom, width, height = [0.123+i%3*0.274, 0.53-i//3*0.42, 0.14, 0.2]
    ax2 = fig.add_axes([left, bottom, width, height])
    ax2.imshow(np.abs(rec[v][350:550,650:850]),cmap='gray',vmax=1.05,vmin=0.95)
    ax2.axis('off')
    
    fig.colorbar(im, cax=cax, orientation='vertical')
plt.savefig(f'rec_abs_{iter}',dpi=450)

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(15,8))
fig.suptitle(f'reconstructed probe amp, iter {iter}', fontsize=15)
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.title('ss')

for i,v in enumerate(['v1','v2','noe','v1_prb_','v2_prb','noe_prb']):
    im=ax[np.unravel_index(i, (2,3))].imshow(np.abs(rec_prb[v]),cmap='gray',vmax=1.55,vmin=0.45)
    divider = make_axes_locatable(ax[np.unravel_index(i, (2,3))])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    ax[np.unravel_index(i, (2,3))].set_title(l[v])
    left, bottom, width, height = [0.123+i%3*0.274, 0.53-i//3*0.42, 0.14, 0.2]
    ax2 = fig.add_axes([left, bottom, width, height])
    ax2.imshow(np.abs(rec_prb[v][350:550,650:850]),cmap='gray',vmax=1.55,vmin=0.45)
    ax2.axis('off')
    
    fig.colorbar(im, cax=cax, orientation='vertical')
plt.savefig(f'prb_abs_{iter}',dpi=450)

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(15,8))
fig.suptitle(f'reconstructed probe angle, iter {iter}', fontsize=15)
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.title('ss')

for i,v in enumerate(['v1','v2','noe','v1_prb_','v2_prb','noe_prb']):
    im=ax[np.unravel_index(i, (2,3))].imshow(np.angle(rec_prb[v]),cmap='gray',vmin=-1.5,vmax=1.5)
    divider = make_axes_locatable(ax[np.unravel_index(i, (2,3))])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    ax[np.unravel_index(i, (2,3))].set_title(l[v])
    left, bottom, width, height = [0.123+i%3*0.274, 0.53-i//3*0.42, 0.14, 0.2]
    ax2 = fig.add_axes([left, bottom, width, height])
    ax2.imshow(np.angle(rec_prb[v][350:550,650:850]),cmap='gray',vmin=-1.5,vmax=1.5)
    ax2.axis('off')
    
    fig.colorbar(im, cax=cax, orientation='vertical')
plt.savefig(f'prb_angle_{iter}',dpi=450)